### Label "1" correpsonds to Doxed Tweets and label "0" is for benign tweets

In [1]:
import pandas as pd
import numpy as np

In [2]:
file_name =  "doxing_dataset.xlsx"   #only doxed tweets

In [3]:
df = pd.read_excel(io=file_name)

In [4]:
df.columns   

Index(['filename', 'tweet', 'Unnamed: 2', 'Unnamed: 3'], dtype='object')

In [5]:
df_class1 = df.drop(['Unnamed: 2', 'Unnamed: 3'], axis = 1)  # remove extra unwanted columns

In [6]:
df_class1['label'] = 1   #doxed tweets df

In [7]:
df_class1.shape

(93, 3)

In [8]:
df_class1.head(1)

,filename,tweet,label
0,1008,@goreadrinette IP Address: XXX.XXX.XX.XX (Loca...,1


In [9]:
### doxed tweets file number stored in skip

In [11]:
skip = list(set(df_class1.filename.tolist()))
skip.extend([575, 493, 45, 686, 389, 496, 96, 1293, 874])  ### corrupted files

In [12]:
### get benign/non-doxing tweets

In [13]:
import json
import os

path = "doxing-samples/"
all_files = os.listdir('doxing-samples/')

In [14]:
list_ = []
for a_file in all_files:
    ff = int(a_file.split('.')[0])
    if ff in skip or ff > 1000:
        continue
        
    f = open(path+str(a_file))
    data = json.load(f)
    
    if 'matching_rules' in data:
        all_tags = data['matching_rules']
        tg = [a_tag['tag'] for a_tag in all_tags][0]
        
    all_twts_len, all_tags_len = 0,0
    if 'includes' in data:
        if 'tweets' in data['includes']:
            all_twts = data['includes']['tweets']        
    list_.append([a_file, [a_twt['text'] for a_twt in all_twts][0], tg])
    
    if 'data' in data:
        if 'text' in data['data']:
            list_.append([a_file, data['data']['text'], tg])
            
            
print("All Done")

df_class0 = pd.DataFrame(list_)

All Done


In [15]:
df_class0.shape

(1854, 3)

In [16]:
df_class0.head(1)    # benign tweets df

,0,1,2
0,153.json,Answers: \n1) Source IP address: 8.8.8.8\n2) D...,ip


In [17]:
df_class0.columns

RangeIndex(start=0, stop=3, step=1)

In [18]:
df_class0 = df_class0.drop([2], axis=1)  # drop PII type

In [19]:
df_class0.shape

(1854, 2)

In [20]:
df_class0.columns = ['filename', 'tweet']

In [21]:
df_class0['label'] = 0

In [24]:
df_class0['filename'] = df_class0['filename'].apply(lambda x: int(x.split('.')[0]))

In [26]:
df_class0 = df_class0.drop_duplicates(subset='tweet', keep="first")  ## drop duplicate tweets
df_class0.shape

(831, 3)

In [27]:
df = pd.concat([df_class0, df_class1]).reset_index(drop=True)  ## concat benign and doxed tweets
df.shape

(924, 3)

In [28]:
df = df.drop(['filename'], axis = 1)   # drop filenames

In [29]:
from sklearn.utils import shuffle

In [30]:
df = shuffle(df)   #randomize the dataframe

In [31]:
def pre_process(tweet):
    tweet = tweet.replace('\n',' ')
    tweet = tweet.lower()
    return tweet

In [32]:
df['tweet'] = df['tweet'].apply(pre_process)   # preprocess the tweets

In [33]:
df.to_pickle("complete_doxing_dataset.pkl")  # save the dataset as a pickle file